# Single Node Single GPU Training

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask

2021-09-24 20:25:49.041056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

2021-09-24 20:25:53.188308: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-24 20:25:53.189252: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-24 20:25:53.233277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-24 20:25:53.233872: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-09-24 20:25:53.233922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-24 20:25:53.235703: I tensorflow/stream_executor/platform/default/dso_loade

1 Physical GPUs, 1 Logical GPUs


In [15]:
def train_model_fit(n_epochs, base_lr, batchsize, classes):
    
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        classes=classes)
        
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='newproj', config=wbargs)
    
    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'datasets/birds/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2).cache().shuffle(1000)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'datasets/birds/valid',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
   
    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    start = time.time()

    model.fit(
        train_ds, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_single/')

SyntaxError: invalid syntax (1840879821.py, line 19)

In [16]:
model_params = {'n_epochs': 50, 
                'base_lr': .02,
                'classes': 285,
               'batchsize': 64}

wbargs = {**model_params,
    'Notes':"tf_v100_2x",
    'Tags': ['single', 'gpu', 'tensorflow'],
    'dataset':"Birds",
    'architecture':"ResNet50"}

In [17]:
tester = train_model_fit(**model_params)

2021-09-24 20:27:45.852312: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/birds/train'

In [14]:
import os
os.getcwd()

'/home/jovyan/git-repos/tensorflow_ds'